In [1]:
file_path = "Bangla-NER-Splitted-Dataset.json"

In [2]:
import json
  
# Opening JSON file
f = open(file_path, mode="r", encoding="utf-8")
  
# returns JSON object as 
# a dictionary
json_data = json.load(f)

In [3]:
json_data.keys()

dict_keys(['test', 'train', 'validation'])

In [4]:
train_json_data = json_data['train']
val_json_data = json_data['validation']
test_json_data = json_data['test']

In [5]:
import pandas as pd
import re
import numpy as np

train_df = pd.DataFrame(columns=['ner_tags', 'tokens'])
val_df = pd.DataFrame(columns=['ner_tags', 'tokens'])
test_df = pd.DataFrame(columns=['ner_tags', 'tokens'])

In [6]:
def read_json_file(json_file, df):
    token_docs = []
    tag_docs = []

    for idx,doc in enumerate(json_file):
        tokens = doc['sentence']
        tags = doc['iob_tags']
        token_docs.append(tokens)
        tag_docs.append(tags)
        df.loc[idx] = pd.Series({'ner_tags':tags, 'tokens':tokens})
    
    return df, token_docs, tag_docs

In [7]:
train_df,_,tag_docs = read_json_file(train_json_data, train_df)

In [8]:
val_df,_,_ = read_json_file(val_json_data, val_df)

In [9]:
test_df,_,_ = read_json_file(test_json_data, val_df)

In [193]:
train_df[:1]

,ner_tags,tokens
0,"[O, O, O, B-PER, B-PER, I-PER, I-PER, O, O, O,...","[ত্রাণ, ও, সমাজকল্যাণ, সম্পাদক, সুজিত, রায়, নন..."


In [194]:
val_df[:1]

,ner_tags,tokens
0,"[O, O, O, B-LOC, O, O, O, O]","[৫%, তার, চাইতে, পশ্চিমোরে, এর, সাক্ষরতার, হার..."


In [195]:
test_df[:1]

,ner_tags,tokens
0,"[O, O, O, B-LOC, O, O, O, O]","[৫%, তার, চাইতে, পশ্চিমোরে, এর, সাক্ষরতার, হার..."


In [196]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

In [197]:
datasets = DatasetDict()
datasets['train'] = train_ds
datasets['validation'] = val_ds
datasets['test'] = test_ds

In [198]:
datasets["train"][3]["tokens"]

['তিনি',
 'বলছিলেন',
 'সবচেয়ে',
 'বড়',
 'কথা',
 'উনি',
 'খুব',
 'ভালো',
 'মানুষ',
 'ছিলেন']

In [199]:
datasets["train"][3]["ner_tags"]

['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O']

In [200]:
# create set from list
expanded_tag_docs = []

for tags in tag_docs:
    for tag in tags:
        expanded_tag_docs.append(tag)

In [201]:
unique_tags = set(expanded_tag_docs)
unique_tags

{'B-LOC', 'B-OBJ', 'B-ORG', 'B-PER', 'I-LOC', 'I-OBJ', 'I-ORG', 'I-PER', 'O'}

In [202]:
def assign_label(examples):
    mapping = {'B-LOC':0, 'B-OBJ':1, 'B-ORG':2, 'B-PER':3, 'I-LOC':4, 'I-OBJ':5, 'I-ORG':6, 'I-PER':7, 'O':8}
    ner_labels = []
    for example in examples["ner_tags"]:
        ner_labels.append(mapping[example])
    examples["ner_labels"] = ner_labels
    return examples

In [203]:
datasets = datasets.map(assign_label)

Map:   0%|          | 0/64155 [00:00<?, ? examples/s]

Map:   0%|          | 0/3565 [00:00<?, ? examples/s]

Map:   0%|          | 0/3565 [00:00<?, ? examples/s]

In [204]:
from transformers import AutoTokenizer

model_checkpoint = "../Bengali Pretraining/models/unigram/unigram-long-text"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Didn't find file ../Bengali Pretraining/models/unigram/unigram-long-text/added_tokens.json. We won't load it.
loading file None
loading file ../Bengali Pretraining/models/unigram/unigram-long-text/special_tokens_map.json
loading file ../Bengali Pretraining/models/unigram/unigram-long-text/tokenizer_config.json
loading file ../Bengali Pretraining/models/unigram/unigram-long-text/tokenizer.json


In [205]:
# num_added_tokens = tokenizer.add_tokens(["-","’","০","১","২","৩","৪","৫","৬","৭","৮","৯"])

In [206]:
inputs = tokenizer(datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '▁ত্রাণ',
 '▁',
 'ও',
 '▁সমাজকল্যাণ',
 '▁সম্পাদক',
 '▁সুজিত',
 '▁রায়',
 '▁নন্দী',
 '▁প্রমুখ',
 '▁সংবাদ',
 '▁সম্মেলনে',
 '▁উপস্থিত',
 '▁ছিলেন',
 '[SEP]']

In [207]:
inputs.word_ids()

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]

In [208]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [209]:
labels = datasets["train"][0]["ner_labels"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[8, 8, 8, 3, 3, 7, 7, 8, 8, 8, 8, 8]
[-100, 8, 8, 8, 8, 3, 3, 7, 7, 8, 8, 8, 8, 8, -100]


In [210]:
datasets["train"][1]

{'ner_tags': ['O',
  'O',
  'B-PER',
  'B-OBJ',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'B-PER',
  'O',
  'B-OBJ',
  'O'],
 'tokens': ['পরিকল্পনা',
  'অনুযায়ী',
  'তারা',
  'বাসায়',
  'ঢুকে',
  'দুই',
  'অতিথিকে',
  'নগ্ন',
  'করে',
  'তাদের',
  'মাঝখানে',
  'এক',
  'ছাত্রীকে',
  'বসিয়ে',
  'ছবি',
  'তোলেন'],
 '__index_level_0__': 1,
 'ner_labels': [8, 8, 3, 1, 8, 8, 3, 8, 8, 3, 8, 8, 3, 8, 1, 8]}

In [211]:
mapping = {'B-LOC':0, 'B-OBJ':1, 'B-ORG':2, 'B-PER':3, 'I-LOC':4, 'I-OBJ':5, 'I-ORG':6, 'I-PER':7, 'O':8}
label_names = list()
for k, v in mapping.items():
    label_names.append(k)

label_names

['B-LOC', 'B-OBJ', 'B-ORG', 'B-PER', 'I-LOC', 'I-OBJ', 'I-ORG', 'I-PER', 'O']

In [212]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, max_length=128
    )
    all_labels = examples["ner_labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [213]:
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

Map:   0%|          | 0/64155 [00:00<?, ? examples/s]

Map:   0%|          | 0/3565 [00:00<?, ? examples/s]

Map:   0%|          | 0/3565 [00:00<?, ? examples/s]

In [214]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 64155
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3565
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3565
    })
})

In [215]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [216]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    8,    8,    8,    8,    3,    3,    7,    7,    8,    8,    8,
            8,    8, -100, -100, -100, -100, -100],
        [-100,    8,    8,    3,    1,    8,    8,    3,    4,    8,    8,    3,
            8,    8,    3,    8,    1,    8, -100]])

In [217]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 8, 8, 8, 8, 3, 3, 7, 7, 8, 8, 8, 8, 8, -100]
[-100, 8, 8, 3, 1, 8, 8, 3, 4, 8, 8, 3, 8, 8, 3, 8, 1, 8, -100]


In [218]:
import evaluate

metric = evaluate.load("seqeval")

In [219]:
labels = datasets["train"][0]["ner_labels"]
labels = [label_names[i] for i in labels]
labels

['O', 'O', 'O', 'B-PER', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O']

In [220]:
predictions = labels.copy()
predictions[2] = 'B-PER'
metric.compute(predictions=[predictions], references=[labels])

{'PER': {'precision': 0.6666666666666666,
  'recall': 1.0,
  'f1': 0.8,
  'number': 2},
 'overall_precision': 0.6666666666666666,
 'overall_recall': 1.0,
 'overall_f1': 0.8,
 'overall_accuracy': 0.9166666666666666}

In [221]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [222]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [223]:
id2label

{0: 'B-LOC',
 1: 'B-OBJ',
 2: 'B-ORG',
 3: 'B-PER',
 4: 'I-LOC',
 5: 'I-OBJ',
 6: 'I-ORG',
 7: 'I-PER',
 8: 'O'}

In [224]:
label2id

{'B-LOC': 0,
 'B-OBJ': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-OBJ': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8}

In [225]:
from transformers import set_seed
set_seed(42)

In [226]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file ../Bengali Pretraining/models/unigram/unigram-long-text/config.json
Model config BertConfig {
  "_name_or_path": "../Bengali Pretraining/models/unigram/unigram-long-text",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-OBJ",
    "2": "B-ORG",
    "3": "B-PER",
    "4": "I-LOC",
    "5": "I-OBJ",
    "6": "I-ORG",
    "7": "I-PER",
    "8": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-OBJ": 1,
    "B-ORG": 2,
    "B-PER": 3,
    "I-LOC": 4,
    "I-OBJ": 5,
    "I-ORG": 6,
    "I-PER": 7,
    "O": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_

In [227]:
# model.resize_token_embeddings(len(tokenizer))

In [228]:
model.config.num_labels

9

In [229]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [230]:
from transformers import TrainingArguments
# batch_size = 16
batch_size = 32

args = TrainingArguments(
    "models/ner",
    report_to = None,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    #save_strategy="epoch",
    learning_rate=2e-5,
    # learning_rate=3e-5,
    #num_train_epochs=3,
    num_train_epochs=4,
    weight_decay=0.01,
    warmup_ratio=0.06,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [231]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [232]:
import transformers
trainer.remove_callback(transformers.integrations.TensorBoardCallback)

In [233]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3565
  Batch size = 32


{'eval_loss': 2.4414865970611572,
 'eval_precision': 0.02632137550413925,
 'eval_recall': 0.10829694323144105,
 'eval_f1': 0.04234972677595628,
 'eval_accuracy': 0.07318426284076374,
 'eval_runtime': 2.9445,
 'eval_samples_per_second': 1210.744,
 'eval_steps_per_second': 38.037}

In [234]:
trainer.train()

***** Running training *****
  Num examples = 64155
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8020


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.281000,0.277574,0.702713,0.696167,0.699425,0.899955
2,0.231600,0.259539,0.723801,0.689665,0.706321,0.905226
3,0.190800,0.265740,0.709305,0.720524,0.714870,0.904671
4,0.167500,0.282348,0.707635,0.721300,0.714402,0.904733


***** Running Evaluation *****
  Num examples = 3565
  Batch size = 32
Saving model checkpoint to models/ner/checkpoint-2005
Configuration saved in models/ner/checkpoint-2005/config.json
Model weights saved in models/ner/checkpoint-2005/pytorch_model.bin
tokenizer config file saved in models/ner/checkpoint-2005/tokenizer_config.json
Special tokens file saved in models/ner/checkpoint-2005/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3565
  Batch size = 32
Saving model checkpoint to models/ner/checkpoint-4010
Configuration saved in models/ner/checkpoint-4010/config.json
Model weights saved in models/ner/checkpoint-4010/pytorch_model.bin
tokenizer config file saved in models/ner/checkpoint-4010/tokenizer_config.json
Special tokens file saved in models/ner/checkpoint-4010/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3565
  Batch size = 32
Saving model checkpoint to models/ner/checkpoint-6015
Configuration saved in models/ner/checkpoint-

TrainOutput(global_step=8020, training_loss=0.23953841996014563, metrics={'train_runtime': 479.8018, 'train_samples_per_second': 534.846, 'train_steps_per_second': 16.715, 'total_flos': 5117780019313440.0, 'train_loss': 0.23953841996014563, 'epoch': 4.0})

In [224]:
trainer.save_model()

Saving model checkpoint to models/ner
Configuration saved in models/ner/config.json
Model weights saved in models/ner/pytorch_model.bin
tokenizer config file saved in models/ner/tokenizer_config.json
Special tokens file saved in models/ner/special_tokens_map.json


In [235]:
# trainer.args.num_train_epochs = trainer.args.num_train_epochs + 1
# trainer.train("models/ner/checkpoint-2005")

In [236]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [237]:
trainer.evaluate(tokenized_datasets["test"])

***** Running Evaluation *****
  Num examples = 3565
  Batch size = 32


{'eval_loss': 0.26574015617370605,
 'eval_precision': 0.7093045471914405,
 'eval_recall': 0.7205240174672489,
 'eval_f1': 0.7148702642853704,
 'eval_accuracy': 0.9046709096793084,
 'eval_runtime': 2.8452,
 'eval_samples_per_second': 1253.006,
 'eval_steps_per_second': 39.365,
 'epoch': 4.0}